# BALTO Graphical User Interface (prototype)

This Jupyter notebook creates a GUI (graphical user interface) for the BALTO (Brokered Alignment of Long-Tail Observations) project.  BALTO is funded by the NSF EarthCube program.  The GUI aims to provide a simplified and customizable method for users to access data sets of interest on servers that support the OpenDAP data access protocol.  This interactive GUI runs within the Jupyter notebook and uses the Python packages: <b>ipywidgets</b> (for widget controls), <b>ipyleaflet</b> (for interactive maps) and <b>pydap</b> (an OpenDAP client).

The Python source code to create the GUI and to process events is in a Python module called <b>balto_gui.py</b> that must be copied into the same directory as this Jupyter notebook.

This is an <b>accordion-style</b> GUI, which allows you to switch between GUI panels without scrolling in the notebook.

You can run this notebook in a browser window without installing anything on your computer, using something called <b>Binder</b>.  On GitHub, you will see a Binder icon and a link labeled "Launch Binder".

To run this Jupyter notebook without Binder, it is recommended to install Python 3.7 from an Anaconda distribution and to then create a <b>conda environment</b> called <b>balto</b>.  Instructions for how to create a conda environment are given in Appendix 1.

#### Table of Contents

1.  <a href="#start_gui">Import and Start the BALTO GUI</a> <br>
2.  <a href="#get_data">Download and Explore Some Data</a> <br>
3.  <a href="#plot_data">Plot Some Data</a> <br>
4.  <a href="#var_match">Matching Variable Names to Standardized Names</a> <br>
5.  <a href="#low_level">Low-level Access to GUI Settings</a> <br>
6.  <a href="#references">References for More Info</a> <br>
7.  <a href="#setup">Appendix 1: Set up a conda environment called "balto"</a> <br>

## Import and Start the BALTO GUI  <a id="start_gui"></a>

In [1]:
import balto_gui as bg
balto = bg.balto_gui()
balto.show_gui()

Output()

New time indices, ti1, ti2 = 1660 1856

Variable saved in: balto.user_var


## Download and Explore Some Data  <a id="get_data"></a>

First open the <b>Browse Data</b> panel.
If you changed the "OpenDAP URL Dir", click the Reset button at the bottom of the Data panel to restore the defaults.  The OpenDAP URL Dir should again be: http://test.opendap.org/dap/data/nc/.

Click on the Go button. <br>
From the <b>Filename droplist</b>, choose <b>sst.mnmean.nc.gz</b>. <br>
From the <b>Variable</b> droplist, choose <b>sst</b>.

Notice that the shape of this dataset is <b>(1857, 89, 180)</b>,
with dimensions: <b>('time', 'lat', 'lon')</b>.

Next, open the <b>Spatial Extent</b> panel and use the interactive map to choose a geographic bounding box for some region you are interested in.  Try choosing different basemaps from the Basemap droplist above the map.  You can also click on the Full Screen icon in the upper right corner to expand the map to the full screen of your computer.

You can also type new bounding lats and lons into the text boxes and click the <b>Update</b> button.  The map will then zoom to show a bounding box that contains your bounding box.  Note, however, that the aspect ratio of the GUI's map window may not match the shape of your bounding box.  

Now let's zoom into the Carribean island of Puerto Rico.  In the text boxes, enter <b>-68.2</b> for the West edge, <b>-64.7</b> for the East edge, <b>18.9</b> for the North edge and <b>17.6</b> for the South edge, then click the <b>Update</b> button.  This should zoom in to a view in which the whole island is visible in the window, with water visible all around.  

Let's now check how the monthly mean sea surface temperature has changed over the years spanned by the SST dataset we downloaded.  To see the date and time range spanned by this dataset, open the <b>Date Range</b> panel.  If you want, you can change the dates and times to restrict the data to a shorter time period.

Next, open the <b>Download Data</b> panel and click on the <b>Download</b> button.  This causes the SST dataset --- restricted to this spatial extent and date range --- to be downloaded to your computer and stored in the <b>balto</b> object as <b>user_var</b>.  This may take a minute or so, before you can execute the next cell.

You can now begin to work with the data in this notebook.


In [9]:
sst = balto.user_var
print('type(sst) =', type(sst) )
print('sst.dtype =', sst.dtype )
print('sst.shape =', sst.shape )
# print(sst)
w = (sst == 32767)
scale_factor = 0.00999999978
sst2 = (sst * scale_factor)
sst2[w] = -9999.0
a = sst2[50,44:50,40:46]
print(a)


type(sst) = <class 'numpy.ndarray'>
sst.dtype = >i2
sst.shape = (196, 89, 180)
[[28.90999936 28.91999936 28.96999936 29.06999936 29.21999936 29.36999935]
 [28.88999936 28.88999936 28.92999936 29.01999936 29.14999936 29.27999936]
 [28.54999937 28.57999937 28.63999937 28.75999937 28.89999936 29.02999936]
 [27.91999939 28.02999938 28.15999938 28.35999938 28.52999937 28.67999937]
 [27.1899994  27.3799994  27.61999939 27.86999939 28.10999938 28.27999938]
 [26.59999941 26.79999941 27.03999941 27.2999994  27.56999939 27.79999939]]


## Plot Some Data  <a id="plot_data"></a>

In [ ]:
# grid = sst[10,:,:]
# balto.show_grid( grid, cmap='rainbow', xsize=8, ysize=8)

## Matching Variable Names to Standardized Names  <a id="var_match"></a>

In [ ]:
# This uses an SVO service at:  http://34.73.227.230:8000/match_phrase/
balto.get_possible_svo_names('sst', SHOW_IRI=False)


## Low-Level Access to the GUI Settings   <a id="low_level"></a>

In [5]:
print(balto.var_short_names)
time = balto.dataset.time
print(time.actual_range)
print(time.units)

['lat', 'lon', 'time', 'time_bnds', 'sst']
[19723.0, 76214.0]
days since 1800-1-1 00:00:00


In [6]:
print(balto.map_minlon_box.value)

-180.0


## References for More Info  <a id="references"></a>

A Good Anaconda Reference

A Good BALTO Reference <br>
https://www.earthcube.org/group/brokered-alignment-long-tail-observations-balto <br>
https://www.nsf.gov/awardsearch/showAward?AWD_ID=1740704

A Good Binder Reference <br>
https://github.com/binder-examples

A Jupyter Notebook Reference

A Good OpenDAP Reference <br>
https://www.opendap.org/ <br>
https://opendap.github.io/documentation/QuickStart.html <br>
https://www.opendap.org/support/user-documentation

<b>NetCDF CF Conventions</b> <br>
http://cfconventions.org/cf-conventions/cf-conventions.html

<b>appmode</b> Jupyter plugin (not used) <br>
https://github.com/oschuett/appmode <br>
https://github.com/binder-examples/appmode

<b>traitlets</b> (used by ipywidgets) <br>
https://traitlets.readthedocs.io/en/stable/

The <b>ipywidgets</b> Python package <br>
https://ipywidgets.readthedocs.io/en/latest/index.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_guide.html

The <b>ipyleaflet</b> Python package <br> 
https://ipyleaflet.readthedocs.io/en/latest/

The <b>pydap</b> Python package <br>
https://www.pydap.org/en/latest/ <br>
https://github.com/pydap/pydap

<b>Jupyter Lab</b> <br>
https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html <br>
https://jupyterlab.readthedocs.io/en/stable/user/extensions.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension<br>

<b>Scientific Variables Ontology</b> (SVO) <br>
http://www.geoscienceontology.org/ <br>
Cite Stoica and Peckham papers.


## Appendix 1:  Set up a conda environment called "balto"    <a id="setup"></a>

To run this Jupyter notebook, it is recommended to use Python 3.7 from an Anaconda distribution and to create a "balto" conda environment with the following commands.  This helps to prevent conflicts with other Python packages you may have installed.  After installing Anaconda, use these commands to create the environment. 

``` bash
% conda update -n base -c defaults conda
% conda create --name balto
% conda activate balto
% conda install -c conda-forge ipywidgets
% conda install -c conda-forge ipyleaflet
% conda install -c conda-forge pydap
```

If you want to use this notebook in Jupyter Lab (vs. Jupyter Notebook), you need to install a few more Python packages and extensions, as follows.

``` bash
% conda install -c conda-forge jupyterlab
% conda install -c conda-forge nb_conda_kernels  # (needed for conda envs)

% conda install -c conda-forge nodejs
% conda install -c conda-forge widgetsnbextension
% jupyter labextension install jupyter-leaflet
% jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

When you want to run the notebook, type "conda activate balto" to activate this environment.  Then change to the directory that contains BALTO_GUI.ipynb and type "jupyter notebook" or "jupyter lab".

Finally, choose this notebook by name, "BALTO_GUI.ipynb", and make sure to choose the kernel called:  Python [conda env:balto].  See the References section at the end for more info.